<a href="https://colab.research.google.com/github/Uttchii/learned-image-compression/blob/main/compressai_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 訓練済み画像圧縮モデルのテスト - CompressAI版

このノートブックでは、**訓練済みの**画像圧縮モデルを使用します。
- **論文**: "Joint Autoregressive and Hierarchical Priors for Learned Image Compression" (NeurIPS 2018)
- **ライブラリ**: CompressAI - 訓練済みモデル付き
- **実行環境**: Google Colab（GPU推奨）

**特徴**:
✅ 実際に画像を圧縮・展開可能
✅ 品質レベル調整可能（1-8）
✅ 実用的な性能評価


In [ ]:
# 1. CompressAIライブラリのインストール
!pip install compressai

# 必要なライブラリをインポート
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import BytesIO
from torchvision import transforms

print("✓ ライブラリのインストールとインポートが完了しました")


In [ ]:
# 2. 環境確認とデバイス設定
print("=== 環境確認 ===")
print(f"PyTorch バージョン: {torch.__version__}")
print(f"torchvision バージョン: {torchvision.__version__}")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用デバイス: {device}")

if torch.cuda.is_available():
    print(f"GPU デバイス: {torch.cuda.get_device_name()}")
    print(f"GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("CPU で実行します（GPUの方が高速です）")


In [ ]:
# 3. 訓練済みモデルの読み込み
from compressai.zoo import mbt2018

print("=== 訓練済みモデルの読み込み ===")

# 品質レベル選択 (1: 低品質/低ビットレート, 8: 高品質/高ビットレート)
quality_level = 3
print(f"品質レベル: {quality_level}/8")

# 訓練済みモデルを読み込み（初回は重みをダウンロード）
print("訓練済みモデルをダウンロード中...")
model = mbt2018(quality=quality_level, pretrained=True, progress=True)
model = model.to(device)
model.eval()

print("✓ 訓練済みモデルの読み込み完了！")
print(f"モデル: {model.__class__.__name__}")

# モデルの基本情報
total_params = sum(p.numel() for p in model.parameters())
print(f"総パラメータ数: {total_params:,}")


In [ ]:
# 4. テスト画像の作成と圧縮・展開
print("=== テスト画像での圧縮・展開テスト ===")

# カラフルなテスト画像を生成
def create_test_image():
    """テスト用のカラフルな画像を作成"""
    test_image = torch.zeros(1, 3, 256, 256)

    # 4つの色付き領域を作成
    test_image[0, 0, :128, :128] = 1.0      # 左上: 赤
    test_image[0, 1, :128, 128:] = 1.0      # 右上: 緑
    test_image[0, 2, 128:, :128] = 1.0      # 左下: 青
    test_image[0, :, 128:, 128:] = 0.5      # 右下: グレー

    # グラデーション効果を追加
    for i in range(256):
        for j in range(256):
            if 100 < i < 156 and 100 < j < 156:
                # 中央部分にグラデーション
                test_image[0, :, i, j] = (i - 100) / 56.0

    return test_image

test_image = create_test_image().to(device)
print(f"テスト画像サイズ: {test_image.shape}")

# 圧縮・展開の実行
print("圧縮・展開を実行中...")
with torch.no_grad():
    # 圧縮
    compressed = model.compress(test_image)
    print(f"圧縮後のデータサイズ: {len(compressed['strings'][0])} bytes")

    # 展開
    decompressed = model.decompress(compressed["strings"], compressed["shape"])
    reconstructed = decompressed["x_hat"]

print(f"復元画像サイズ: {reconstructed.shape}")
print("✓ 圧縮・展開が成功しました！")


In [ ]:
# 5. 結果の可視化と品質評価
print("=== 圧縮結果の可視化 ===")

# 圧縮前後の比較
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 元画像
original_img = test_image[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[0].imshow(original_img)
axes[0].set_title('元画像', fontsize=14, fontweight='bold')
axes[0].axis('off')

# 復元画像
reconstructed_img = reconstructed[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[1].imshow(reconstructed_img)
axes[1].set_title('復元画像（訓練済みモデル）', fontsize=14, fontweight='bold')
axes[1].axis('off')

# 差分画像
diff = torch.abs(test_image[0] - reconstructed[0]).cpu().permute(1, 2, 0)
diff_normalized = diff / (diff.max() + 1e-8)  # 正規化
axes[2].imshow(diff_normalized)
axes[2].set_title('差分画像（明るい部分=差が大きい）', fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 品質評価
print("=== 品質評価 ===")
mse = torch.mean((test_image - reconstructed) ** 2).item()
psnr = -10 * torch.log10(torch.tensor(mse)) if mse > 0 else float('inf')

# ビット率計算 (概算)
original_size = test_image.numel() * 8  # 8 bits per pixel (RGB)
compressed_size = len(compressed['strings'][0]) * 8  # bytes to bits
compression_ratio = original_size / compressed_size
bits_per_pixel = compressed_size / (256 * 256)

print(f"MSE: {mse:.6f}")
print(f"PSNR: {psnr:.2f} dB")
print(f"圧縮率: {compression_ratio:.1f}x")
print(f"ビット/ピクセル: {bits_per_pixel:.3f} bpp")
print(f"元データサイズ: {original_size // 8:,} bytes")
print(f"圧縮後サイズ: {len(compressed['strings'][0]):,} bytes")


In [ ]:
# 6. 実際の画像での試行
print("=== 実際の画像での圧縮テスト ===")

# インターネットからサンプル画像をダウンロード
def download_sample_image():
    """サンプル画像をダウンロード"""
    try:
        # Wikipedia の高品質な画像を使用
        url = "https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Vd-Orig.png/256px-Vd-Orig.png"
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        print("✓ サンプル画像をダウンロードしました")
        return img
    except Exception as e:
        print(f"ダウンロードエラー: {e}")
        print("代替画像を生成します...")
        # 代替画像を生成
        img = Image.new('RGB', (256, 256), color='white')
        return img

# 画像の前処理
def preprocess_image(img):
    """画像を前処理してテンソルに変換"""
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    return transform(img).unsqueeze(0)

# サンプル画像での実験
sample_img = download_sample_image()
real_image = preprocess_image(sample_img).to(device)
print(f"実画像サイズ: {real_image.shape}")

# 圧縮・展開
print("実画像を圧縮・展開中...")
with torch.no_grad():
    compressed_real = model.compress(real_image)
    decompressed_real = model.decompress(compressed_real["strings"], compressed_real["shape"])
    reconstructed_real = decompressed_real["x_hat"]

print("✓ 実画像での圧縮・展開が成功しました！")


In [ ]:
# 7. 実画像結果の比較
print("=== 実画像の圧縮結果 ===")

# 比較表示
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 元の実画像
real_original = real_image[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[0].imshow(real_original)
axes[0].set_title('元の実画像', fontsize=14, fontweight='bold')
axes[0].axis('off')

# 復元された実画像
real_reconstructed = reconstructed_real[0].cpu().permute(1, 2, 0).clamp(0, 1)
axes[1].imshow(real_reconstructed)
axes[1].set_title('圧縮・復元後', fontsize=14, fontweight='bold')
axes[1].axis('off')

# 差分
real_diff = torch.abs(real_image[0] - reconstructed_real[0]).cpu().permute(1, 2, 0)
real_diff_normalized = real_diff / (real_diff.max() + 1e-8)
axes[2].imshow(real_diff_normalized)
axes[2].set_title('差分画像', fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 実画像での品質評価
print("=== 実画像での品質評価 ===")
real_mse = torch.mean((real_image - reconstructed_real) ** 2).item()
real_psnr = -10 * torch.log10(torch.tensor(real_mse)) if real_mse > 0 else float('inf')

real_original_size = real_image.numel() * 8
real_compressed_size = len(compressed_real['strings'][0]) * 8
real_compression_ratio = real_original_size / real_compressed_size
real_bpp = real_compressed_size / (256 * 256)

print(f"実画像 MSE: {real_mse:.6f}")
print(f"実画像 PSNR: {real_psnr:.2f} dB")
print(f"実画像 圧縮率: {real_compression_ratio:.1f}x")
print(f"実画像 ビット/ピクセル: {real_bpp:.3f} bpp")
print(f"実画像 圧縮後サイズ: {len(compressed_real['strings'][0]):,} bytes")


In [ ]:
# 8. 品質レベル比較（オプション）
print("=== 異なる品質レベルでの比較 ===")

# 複数の品質レベルで圧縮を試行
quality_levels = [1, 3, 5, 8]  # 低品質から高品質まで
results = []

for q in quality_levels:
    print(f"品質レベル {q} をテスト中...")

    # モデルを読み込み
    temp_model = mbt2018(quality=q, pretrained=True, progress=False)
    temp_model = temp_model.to(device)
    temp_model.eval()

    # テスト画像で圧縮
    with torch.no_grad():
        temp_compressed = temp_model.compress(test_image)
        temp_decompressed = temp_model.decompress(temp_compressed["strings"], temp_compressed["shape"])
        temp_reconstructed = temp_decompressed["x_hat"]

    # 品質評価
    temp_mse = torch.mean((test_image - temp_reconstructed) ** 2).item()
    temp_psnr = -10 * torch.log10(torch.tensor(temp_mse)) if temp_mse > 0 else float('inf')
    temp_size = len(temp_compressed['strings'][0])
    temp_bpp = temp_size * 8 / (256 * 256)

    results.append({
        'quality': q,
        'mse': temp_mse,
        'psnr': temp_psnr,
        'size': temp_size,
        'bpp': temp_bpp
    })

# 結果の表示
print("\n=== 品質レベル比較結果 ===")
print("品質 | PSNR(dB) | ファイルサイズ(bytes) | ビット/ピクセル")
print("-" * 55)
for r in results:
    print(f"{r['quality']:^4} | {r['psnr']:^8.2f} | {r['size']:^17,} | {r['bpp']:^12.3f}")

print("\n💡 解釈:")
print("- 品質レベルが高いほど、PSNRは高く（画質良）、ファイルサイズは大きい")
print("- 品質レベルが低いほど、PSNRは低く（画質劣）、ファイルサイズは小さい")
print("- 用途に応じて適切な品質レベルを選択可能")


In [ ]:
## まとめ

### ✅ 実現できたこと
- **訓練済みモデル**による実際の画像圧縮・展開
- **品質評価**: PSNR、圧縮率、ビット/ピクセルの測定
- **視覚的比較**: 元画像、復元画像、差分画像の表示
- **品質レベル調整**: 1-8の品質レベルでの比較

### 🔬 技術的詳細
- **論文**: "Joint Autoregressive and Hierarchical Priors for Learned Image Compression" (NeurIPS 2018)
- **実装**: CompressAI ライブラリ
- **モデル**: mbt2018 (Minnen, Ballé, Toderici 2018)
- **特徴**: 自己回帰・階層事前分布による高効率圧縮

### 📈 期待される結果
- **PSNR**: 通常 25-45 dB（品質レベルに依存）
- **圧縮率**: 10-100倍（品質レベルに依存）
- **ビット/ピクセル**: 0.1-2.0 bpp

### 🚀 次のステップ
1. **異なる画像での実験**: 自然画像、アニメ、テキストなど
2. **他のモデルとの比較**: JPEG、WebP、他の学習ベース手法
3. **カスタム画像での試行**: アップロード機能の追加
4. **動画圧縮**: CompressAIの動画圧縮モデルの試行

### 💡 応用可能性
- **Webサービス**: 画像配信の高効率化
- **モバイルアプリ**: ストレージ容量の削減
- **研究**: より高性能な圧縮アルゴリズムの開発基盤
